## Imports

In [1]:
from datetime import datetime
import math
import time

import numpy as np
import tensorflow as tf
import train_utils

# Imports from example deep dream notebook
import os
from io import BytesIO
import numpy as np
from functools import partial
import PIL.Image
from IPython.display import clear_output, Image, display, HTML

# Imports for audio reconstruction
import librosa
from IPython.lib.display import Audio


## Declare Flags/Constants

In [2]:
class flags(object):
    def __init__(self):
        self.checkpoint_dir = "/tmp/cifar10_train"
        self.num_examples = 10000
        self.run_once = False
        self.eval_interval_secs = 10
        self.eval_dir = '/tmp/cifar10_eval'
        self.num_subsamples = 301
        self.num_coeffs = 12
        self.train_data = "../../small_mfcc_data.h5"
        self.batch_size = 50
        self.checkpoint_step = 30000

FLAGS = flags()
tf.app.FLAGS = FLAGS

SR = 22050
RECON_SHAPE = 662077
HOP_LENGTH = int(500 * (1325. / 301.))

## Cifar10.py Code Contents
Copied so that we can manually set our own flags

In [3]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Builds the CIFAR-10 network.

Summary of available functions:

 # Compute input images and labels for training. If you would like to run
 # evaluations, use inputs() instead.
 inputs, labels = distorted_inputs()

 # Compute inference on the model inputs to make a prediction.
 predictions = inference(inputs)

 # Compute the total loss of the prediction with respect to the labels.
 loss = loss(predictions, labels)

 # Create a graph to run one step of training with respect to the loss.
 train_op = train(loss, global_step)
"""
# pylint: disable=missing-docstring
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gzip
import os
import pandas as pd
import re
import sys
import tarfile
import numpy as np
import train_utils

from six.moves import urllib
import tensorflow as tf
import tflearn

from tensorflow.models.image.cifar10 import cifar10_input

# NOTE: Commented-out so that we can use hard-coded flags instead
# FLAGS = tf.app.flags.FLAGS

NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = cifar10_input.NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = cifar10_input.NUM_EXAMPLES_PER_EPOCH_FOR_EVAL

# Some network parameters
# TODO(smurching): Make these flags?
NUM_CONV_LAYERS = 5
NUM_RECC_LAYERS = 2
CONV_FILTER_SIZE = [5, 5]
RECC_LAYER_SIZE = 30

# Constants describing the training process.
MOVING_AVERAGE_DECAY = 0.9999     # The decay to use for the moving average.
NUM_EPOCHS_PER_DECAY = 350.0      # Epochs after which learning rate decays.
LEARNING_RATE_DECAY_FACTOR = 0.1  # Learning rate decay factor.
INITIAL_LEARNING_RATE = 0.001       # Initial learning rate.


def build_conv_layers(input_tensor, num_layers, filter_size=None):
  '''
  Returns a tensor corresponding to <num_layers> connected convolutional layers 
  with max pooling between layers. The first convolutional layer takes 
  input_tensor as its input.

  filter_size: shape (along the [num_subsamples, num_coeffs] axes) of filter
  '''

  # Set filter size to a default if none is specified
  if filter_size is None:
    filter_size = [2, 2]

  final_layer = input_tensor
  for i in xrange(num_layers):
    # Expects input tensor [batch_size, height, width, in_channels]
    # These dimensions correspond to [batch_size, num_subsamples, num_coeffs, 1] in our case
    with tf.variable_scope('conv_%d'%(i + 1)) as scope:
      conv = tflearn.layers.conv.conv_2d (final_layer, nb_filter=1, filter_size=filter_size, strides=1,
        padding='same', activation='linear', bias=True, weights_init='uniform_scaling',
        bias_init='zeros', regularizer=None, weight_decay=0.001,
        restore=True, reuse=None, scope=scope)

    # TODO(smurching): Intelligently pick values for the kernel size/stride here
    final_layer = tflearn.layers.conv.max_pool_2d (conv, kernel_size=[1, 3, 3, 1],
      strides=[1, 2, 2, 1], padding='same', name='MaxPool2D_%d'%i)
  return final_layer

def build_recurrent_layers(input_tensor, num_layers, units_per_layer=3, activation='sigmoid', dropout=0.8):
  final_layer = input_tensor
  for i in xrange(num_layers):
    is_last_layer = (i == num_layers - 1)
    if is_last_layer:
      # We don't apply softmax for the last layer because 
      # tf.nn.sparse_softmax_cross_entropy_with_logits accepts the unscaled logits 
      # and performs the softmax internally for efficiency.      
      curr_activation = 'linear'      
    else:
      curr_activation = activation      

    with tf.variable_scope('recurrent_%d'%(i + 1)) as scope:
      # TODO(smurching): Pick dropout probability more intelligently, currently just a random guess
      # Get output of recurrent layer as a <timesteps>-length list of prediction tensors of shape
      # [batch_size, num_units] if this isn't our final recurrent layer. Otherwise, just get a single 2D
      # output tensor of shape [batch_size, num_units]
      with tf.device("/cpu:0"):
        final_layer = tflearn.layers.recurrent.lstm(final_layer, n_units=units_per_layer, scope=scope,
          reuse=None, activation=curr_activation, dropout=dropout, return_seq=(not is_last_layer))
      if not is_last_layer:
        final_layer = tf.pack(final_layer, axis=1)
      
  return final_layer


def inference(songs):
  """Build the CIFAR-10 model.

  Args:
    songs: MFCC data (numpy array of shape [batch_size, num_subsamples, num_coeffs, 1]) 
    returned from train_utils.inputs().


  Returns:
    Logits.

  """
  # We instantiate all variables using tf.get_variable() instead of
  # tf.Variable() in order to share variables across multiple GPU training runs.
  # If we only ran this model on a single GPU, we could simplify this function
  # by replacing all instances of tf.get_variable() with tf.Variable().
  #
  print("Flags num examples: %s"%FLAGS.num_examples)
  print("Training network with %s convolutional layers, %s recurrent layers using learning rate %s"%(
    NUM_CONV_LAYERS, NUM_RECC_LAYERS, INITIAL_LEARNING_RATE))
  print("Convolutional layer filter size: %s"%CONV_FILTER_SIZE)
  print("Each recurrent layer has %s units"%RECC_LAYER_SIZE)

  convolutional_layers = build_conv_layers(songs, num_layers=NUM_CONV_LAYERS,
    filter_size=CONV_FILTER_SIZE)
  conv_reshaped = tf.squeeze(convolutional_layers, squeeze_dims=[3])
  print("RNN input shape (batch_size x timesteps x num_coeffs): %s"%conv_reshaped.get_shape())

  return build_recurrent_layers(conv_reshaped, num_layers=NUM_RECC_LAYERS,
    units_per_layer=RECC_LAYER_SIZE, activation='sigmoid')


In [4]:
READER = train_utils.HDF5BatchProcessor(filename=FLAGS.train_data,
  batch_size=FLAGS.batch_size)

graph = tf.Graph()
session = tf.Session(graph=graph)

## Helper Functions

In [5]:
def restore_graph(sess, saver):
    ckpt = tf.train.get_checkpoint_state(FLAGS.checkpoint_dir)
    if ckpt and ckpt.model_checkpoint_path:
      # Restores from checkpoint
      if FLAGS.checkpoint_step is None:
          checkpoint_path = ckpt.model_checkpoint_path
      else:
          checkpoint_filename = "model.ckpt-%s"%FLAGS.checkpoint_step
          checkpoint_path = os.path.join(FLAGS.checkpoint_dir, checkpoint_filename)
      print("Restoring variables from checkpoint file %s"%checkpoint_path)
      saver.restore(sess, checkpoint_path)
      # Assuming model_checkpoint_path looks something like:
      #   /my-favorite-path/cifar10_train/model.ckpt-0,
      # extract global_step from it.
      global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
      return True  
    print('No checkpoint file found')
    return False

def get_saver():
    # Restore the moving average version of the learned variables for eval.
    variable_averages = tf.train.ExponentialMovingAverage(
        MOVING_AVERAGE_DECAY)
    variables_to_restore = variable_averages.variables_to_restore()
    saver = tf.train.Saver(variables_to_restore)
    return saver

def build_feed_dict(reader, features_placeholder, label_placeholder):
  result = {}
  features, labels = train_utils.inputs(reader)
  result[features_placeholder] = features
  result[label_placeholder] = labels
  return result

def add_inference_nodes(graph):
  """Add inference nodes to our graph"""
  with graph.as_default():
    # Get feature and label placeholders
    features_placeholder = tf.placeholder(tf.float32, shape=[None,
      FLAGS.num_subsamples, FLAGS.num_coeffs, 1])
    label_placeholder = tf.placeholder(tf.int32, shape=[None,])

    # Build a Graph that computes the logits predictions from the
    # inference model.
    logits = inference(features_placeholder)

In [6]:
add_inference_nodes(graph)

Flags num examples: 10000
Training network with 5 convolutional layers, 2 recurrent layers using learning rate 0.001
Convolutional layer filter size: [5, 5]
Each recurrent layer has 30 units
RNN input shape (batch_size x timesteps x num_coeffs): (?, 10, 1)


In [7]:
layer_names = [op.name for op in graph.get_operations()]
conv_layers = [op.name for op in graph.get_operations() if op.type=='Conv2D']
recc_layers = [op.name for op in graph.get_operations() if "recur" in op.name.lower() and "add" in op.name.lower()]
print(recc_layers[:10])


[u'recurrent_1/recurrent_1/recurrent_1/cond/dropout/add', u'recurrent_1/recurrent_1/recurrent_1/BasicLSTMCell/add', u'recurrent_1/recurrent_1/recurrent_1/BasicLSTMCell/add_1/y', u'recurrent_1/recurrent_1/recurrent_1/BasicLSTMCell/add_1', u'recurrent_1/recurrent_1/recurrent_1/BasicLSTMCell/add_2', u'recurrent_1/recurrent_1/recurrent_1/cond_1/dropout/add', u'recurrent_1/recurrent_1/recurrent_1/cond_2/dropout/add', u'recurrent_1/recurrent_1/recurrent_1/BasicLSTMCell_1/add', u'recurrent_1/recurrent_1/recurrent_1/BasicLSTMCell_1/add_1/y', u'recurrent_1/recurrent_1/recurrent_1/BasicLSTMCell_1/add_1']


In [8]:
print(len(conv_layers))
print(conv_layers)

print(filter(lambda x: "lace" in x, layer_names))

5
[u'conv_1/conv_1/Conv2D', u'conv_2/conv_2/Conv2D', u'conv_3/conv_3/Conv2D', u'conv_4/conv_4/Conv2D', u'conv_5/conv_5/Conv2D']
[u'Placeholder', u'Placeholder_1']


To take a glimpse into the kinds of patterns that the network learned to recognize, we will try to generate images that maximize the sum of activations of particular channel of a particular convolutional layer of the neural network. The network we explore contains many convolutional layers, each of which outputs one (lol) feature channel, so we have plenty of patterns to explore.

In [9]:
layers = [op.name for op in graph.get_operations() if op.type=='Conv2D']
feature_nums = [int(graph.get_tensor_by_name(name+':0').get_shape()[-1]) for name in layers]

print('Number of layers', len(layers))
print('Total number of feature channels:', sum(feature_nums))


# Helper functions for TF Graph visualization
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = bytes("<stripped %d bytes>"%size)
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def
  
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

# Visualizing the network graph. Be sure expand the "mixed" nodes to see their 
# internal structure. We are going to visualize "Conv2D" nodes.
tmp_def = rename_nodes(graph.as_graph_def(), lambda s:"/".join(s.split('_',1)))
show_graph(tmp_def)

Number of layers 5
Total number of feature channels: 5


## Naive feature visualization
Let's start with a naive way of visualizing these. Sound-space gradient ascent!

In [10]:
# Check shape of song-batch placeholder for inference graph 
#(should be [batch_size, num_subsamples, num_coeffs, 1])
graph.get_tensor_by_name('Placeholder:0')

<tf.Tensor 'Placeholder:0' shape=(?, 301, 12, 1) dtype=float32>

## Helpers for audio reconstruction

In [11]:
# start with a gray image with a little noise
song_noise = np.random.uniform(size=(301,12,1)) + 5.0

def invlogamplitude(S):
    '''librosa.logamplitude is actually 10*log10, so invert that.'''
    return 10.0 ** (S / 10.0)

def reconstruct_audio(raw_dream_output):
    '''
    Reconstructs audio from the feature representation of a song.
    '''
    dream_output = np.transpose(np.squeeze(raw_dream_output, axis=2))
    recon = reconstruct_audio_helper(dream_output)
    # Listen to the reconstruction
    return Audio(recon, rate=SR)

def reconstruct_audio_helper(mfccs):
    '''
    Reconstructs audio from an MFCC matrix representing a song.
    
    Arguments:
    mfccs: 2D array of shape [num_coeffs, num_subsamples]
    '''
    # Build reconstruction mappings.
    n_mfcc = mfccs.shape[0]
    n_mel = 128
    dctm = librosa.filters.dct(n_mfcc, n_mel)
    n_fft = 2048
    mel_basis = librosa.filters.mel(SR, n_fft)
    # Empirical scaling of channels to get ~flat amplitude mapping.
    bin_scaling = 1.0 / np.maximum(0.0005, np.sum(np.dot(mel_basis.T, mel_basis), axis=0))

    # Reconstruct the approximate STFT squared-magnitude from the MFCCs.
    recon_stft = bin_scaling[:, np.newaxis] * np.dot(mel_basis.T, 
            invlogamplitude(np.dot(dctm.T, mfccs)))


    # Impose reconstructed magnitude on white noise STFT.
    excitation = np.random.randn(RECON_SHAPE)
    E = librosa.stft(excitation, hop_length=HOP_LENGTH)
    recon = librosa.istft(E / np.abs(E) * np.sqrt(recon_stft))
    return recon

## Gradient-ascent functions

In [12]:
# Picking some internal layer. Note that we use outputs before applying the ReLU nonlinearity
# to have non-zero gradients for features with negative initial activations.
layer = 'conv_3/conv_3/BiasAdd'
graph.get_tensor_by_name("%s:0"%layer)
channel = 0 # picking some feature channel to visualize

def T(layer):
    '''Helper for getting layer output tensor'''
    return graph.get_tensor_by_name("%s:0"%layer)

def expand_song(song):
    '''
    Inserts a dimension into the song's feature representation
    to turn it into a single-row batch that can be processed by our inference graph.
    '''
    return np.expand_dims(song, axis=0)

def render_naive(t_obj, img0=song_noise, iter_n=20, step=1.0):
    with graph.as_default():
        with tf.Session() as sess:
            saver = get_saver()
            restore_graph(sess, saver)
            placeholder = T('Placeholder')
            t_score = tf.reduce_mean(t_obj) # defining the optimization objective
            t_grad = tf.gradients(t_score, placeholder)[0] # behold the power of automatic differentiation!

            img = img0.copy()
            for i in range(iter_n):
                g, score = sess.run([t_grad, t_score], {placeholder:expand_song(img)})
                # normalizing the gradient, so the same step size should work 
                g /= g.std()+1e-8         # for different layers and networks
                img += np.squeeze(g*step, axis=0)
                # print(score, end = ' ')
            clear_output()
            # showarray(visstd(img))
            return reconstruct_audio(img)

## Comparing deep dream input & output
We can compare the audio reconstructions of the initial noise we pass to our deep dream
function and the "dreamed" result:

In [13]:
# Deep dream input
reconstruct_audio(song_noise)

In [14]:
# Deep dream output
render_naive(T(layer)[:,:,:,channel])

## Multiscale Song Generation

In [15]:
def tffunc(*argtypes):
    '''Helper that transforms TF-graph generating function into a regular one.
    See "resize" function below.
    '''
    placeholders = list(map(tf.placeholder, argtypes))
    def wrap(f):
        out = f(*placeholders)
        def wrapper(*args, **kw):
            return out.eval(dict(zip(placeholders, args)), session=kw.get('session'))
        return wrapper
    return wrap

# Helper function that uses TF to resize an image
def resize(img, size):
    img = tf.expand_dims(img, 0)
    return tf.image.resize_bilinear(img, size)[0,:,:,:]
resize = tffunc(np.float32, np.int32)(resize)


def calc_grad_tiled(img, t_grad, placeholder, sess, tile_size=512):
    '''Compute the value of tensor t_grad over the image in a tiled way.
    Random shifts are applied to the image to blur tile boundaries over 
    multiple iterations.'''
    sz = tile_size
    h, w = img.shape[:2]
    sx, sy = np.random.randint(sz, size=2)
    img_shift = np.roll(np.roll(img, sx, 1), sy, 0)
    grad = np.zeros_like(img)
    for y in range(0, max(h-sz//2, sz),sz):
        for x in range(0, max(w-sz//2, sz),sz):
            sub = img_shift[y:y+sz,x:x+sz]
            g = sess.run(t_grad, {placeholder:expand_song(sub)})
            grad[y:y+sz,x:x+sz] = g
    return np.roll(np.roll(grad, -sx, 1), -sy, 0)

def render_multiscale(t_obj, img0=song_noise, iter_n=10, step=1.0, octave_n=3, octave_scale=1.4):
    with graph.as_default():
        with tf.Session() as sess:
            saver = get_saver()
            restore_graph(sess, saver)    
            placeholder = T('Placeholder')            
            t_score = tf.reduce_mean(t_obj) # defining the optimization objective
            t_grad = tf.gradients(t_score, placeholder)[0] # behold the power of automatic differentiation!

            img = img0.copy()
            for octave in range(octave_n):
                if octave>0:
                    hw = np.float32(img.shape[:2])*octave_scale
                    img = resize(img, np.int32(hw))
                for i in range(iter_n):
                    g = calc_grad_tiled(img, t_grad, placeholder, sess)
                    # normalizing the gradient, so the same step size should work 
                    g /= g.std()+1e-8         # for different layers and networks
                    img += g*step
                    print('.', end = ' ')
                clear_output()
                # showarray(visstd(img))
                return reconstruct_audio(img)


In [16]:
render_multiscale(T(layer)[:,:,:,channel])

<a id="laplacian"></a>
## Laplacian Pyramid Gradient Normalization

This looks better, but the resulting images mostly contain high frequencies. Can we improve it? One way is to add a smoothness prior into the optimization objective. This will effectively blur the image a little every iteration, suppressing the higher frequencies, so that the lower frequencies can catch up. This will require more iterations to produce a nice image. Why don't we just boost lower frequencies of the gradient instead? One way to achieve this is through the [Laplacian pyramid](https://en.wikipedia.org/wiki/Pyramid_%28image_processing%29#Laplacian_pyramid) decomposition. We call the resulting technique _Laplacian Pyramid Gradient Normalization_.

In [17]:
k = np.float32([1,4,6,4,1])
k = np.outer(k, k)
# TODO(smurching): Check code below, not sure if the modification I made
# was correct.
# k5x5 = k[:,:,None,None]/k.sum()*np.eye(3, dtype=np.float32)
num_channels = 1
k5x5 = k[:,:,None,None]/k.sum()*np.eye(num_channels, dtype=np.float32)


def lap_split(img):
    '''Split the image into lo and hi frequency components'''
    with tf.name_scope('split'):
        lo = tf.nn.conv2d(img, k5x5, [1,2,2,1], 'SAME')
        lo2 = tf.nn.conv2d_transpose(lo, k5x5*4, tf.shape(img), [1,2,2,1])
        hi = img-lo2
    return lo, hi

def lap_split_n(img, n):
    '''Build Laplacian pyramid with n splits'''
    levels = []
    for i in range(n):
        img, hi = lap_split(img)
        levels.append(hi)
    levels.append(img)
    return levels[::-1]

def lap_merge(levels):
    '''Merge Laplacian pyramid'''
    img = levels[0]
    for hi in levels[1:]:
        with tf.name_scope('merge'):
            img = tf.nn.conv2d_transpose(img, k5x5*4, tf.shape(hi), [1,2,2,1]) + hi
    return img

def normalize_std(img, eps=1e-10):
    '''Normalize image by making its standard deviation = 1.0'''
    with tf.name_scope('normalize'):
        std = tf.sqrt(tf.reduce_mean(tf.square(img)))
        return img/tf.maximum(std, eps)

def lap_normalize(img, scale_n=4):
    '''Perform the Laplacian pyramid normalization.'''
    img = tf.expand_dims(img,0)
    tlevels = lap_split_n(img, scale_n)
    tlevels = list(map(normalize_std, tlevels))
    out = lap_merge(tlevels)
    return out[0,:,:,:]

# Showing the lap_normalize graph with TensorBoard
lap_graph = tf.Graph()
with lap_graph.as_default():
    lap_in = tf.placeholder(np.float32, name='lap_in')
    lap_out = lap_normalize(lap_in)
show_graph(lap_graph)

In [18]:
def render_lapnorm(t_obj, img0=song_noise, dispfunc=reconstruct_audio,
                   iter_n=10, step=1.0, octave_n=3, octave_scale=1.4, lap_n=4):
    
    with graph.as_default():
        with tf.Session() as sess:
            saver = get_saver()
            restore_graph(sess, saver) 
            placeholder = T('Placeholder')            
            t_score = tf.reduce_mean(t_obj) # defining the optimization objective
            t_grad = tf.gradients(t_score, placeholder)[0] # behold the power of automatic differentiation!
            # build the laplacian normalization graph
            lap_norm_func = tffunc(np.float32)(partial(lap_normalize, scale_n=lap_n))

            img = img0.copy()
            for octave in range(octave_n):
                if octave>0:
                    hw = np.float32(img.shape[:2])*octave_scale
                    img = resize(img, np.int32(hw))
                for i in range(iter_n):
                    g = calc_grad_tiled(img, t_grad, placeholder, sess)
                    g = lap_norm_func(g)
                    img += g*step
                    print('.', end = ' ')
                clear_output()
                return dispfunc(img)

## Comparing all reconstructions
We can now compare the audio reconstructions we obtain from our various deep-dream methods:

In [33]:
conv_layer_id = 2
layer = 'conv_%s/conv_%s/BiasAdd'%(conv_layer_id, conv_layer_id)

In [34]:
reconstruct_audio(song_noise)

In [38]:
render_naive(T(layer)[:,:,:,channel], iter_n=40)

In [36]:
render_multiscale(T(layer)[:,:,:,channel], iter_n=10)

In [37]:
render_lapnorm(T(layer)[:,:,:,channel], iter_n=10)

## Warping existing songs
We can also warp existing songs

In [24]:
def song_to_mfcc(mp3_filename, n_mfcc=FLAGS.num_coeffs):
    '''
    Returns MFCC representation of passed-in song as a 3D numpy array
    of shape [num_subsamples, num_coeffs, 1]
    '''
    # Load input audio.
    y, sr = librosa.load(mp3_filename)
    print(sr)

    # Calculate mfccs.
    Y = librosa.stft(y)
    # Load MFCCs as a 2D numpy array of size [num_coeffs, num_subsamples]
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=HOP_LENGTH)
    print('mfccs shape: {}'.format(mfccs.shape))
    
    mfcc_slice = mfccs[:,:FLAGS.num_subsamples]
    return np.expand_dims(np.transpose(mfcc_slice), axis=2)


def warp_existing(mp3_filename, iter_n=10):
    song_feature_repr = song_to_mfcc(mp3_filename)
    # Uncomment to play reconstruction of unwarped MFCC data
    # return reconstruct_audio(song_feature_repr)
    return render_naive(T(layer)[:,:,:,channel], song_feature_repr, iter_n=iter_n)

warp_existing("/Users/siddharth/Downloads/the-wall.mp3", iter_n=10)